In [5]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import cx_Oracle
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [6]:
def oneHourAgoFromNow():
    now = datetime.datetime.now()
    nowDatetime = now.strftime('%Y-%m-%d %H:%M')
    later = nowDatetime[:11] + minusHour(nowDatetime[11:13]) + ':00'
    return later

def switch_time(ref):
    if int(ref)%10==int(ref):
        return oneHourAgoFromNow()[:11]+'0'+ref+':00'
    else:
        return oneHourAgoFromNow()[:11]+ref+':00'
    
def minusHour(hour):
    number = int(hour)
    if number==0:
        return "23"
    else:
        number -= 1
        if number%10==number:
            return "0"+str(number)
        else:
            return str(number)

In [52]:
def make_jr_plot():
    reftime = int(oneHourAgoFromNow()[11:13])
    hours = [i for i in range(reftime+4)]
    conn = cx_Oracle.connect('SCOTT/TIGER@localhost:1521/XE')
    db = conn.cursor()
    mdata_list = []
    cmdata_list = []
    pred_list = []
    for hour in range(reftime+1):
        each_time = switch_time(str(hour))
        db.execute('select * from realtimePm10 where 집계시간=:time', time = each_time)
        for record in db:
            mdata_list.append(record)
        db.execute('select * from realtimePm25 where 집계시간=:time', time = each_time)
        for record in db:
            cmdata_list.append(record)
    db.execute('select * from predictedData where reftime=:time and location in :loc', time = oneHourAgoFromNow(), loc='종로구')
    for record in db:
        pred_list.append(record)     
    db.close()
    conn.close()
    
    df = pd.DataFrame(mdata_list, columns=['집계시간','강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구'])
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')
    cdf = pd.DataFrame(cmdata_list, columns=['집계시간','강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구'])
    cdf = cdf.fillna(method='ffill')
    cdf = cdf.fillna(method='bfill')


    mList = list(df['종로구'])
    print(mList)
    mList.extend([pred_list[0][2], pred_list[0][4], pred_list[0][6]])
    cmList = list(cdf['종로구'])
    cmList.extend([pred_list[0][3], pred_list[0][5], pred_list[0][7]])
    
    print(hours)
    print(mList)
    print(cmList)
    
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots()

    ax.plot(hours, mList, label='미세먼지')
    ax.plot(hours, cmList, label='초미세먼지')
    ax.legend(loc=0)
    plt.axvline(x=reftime, color='r', linestyle=':', linewidth=2)
    
    word = oneHourAgoFromNow().replace("-", "")
    word = word.replace(" ", "")
    word = word.replace(":", "")
    title = 'C:\\0.bigData\\4.web\\Soup\\WebContent\\images' + word + '_.png'
    plt.savefig(title)

In [53]:
make_jr_plot()

[24, 28, 28, 29, 28, 30, 30, 31]


IndexError: list index out of range

In [33]:
reftime = int(oneHourAgoFromNow()[11:13])
list1 = [i for i in range(reftime+3)]
list1

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]